In [16]:
import pandas as  pd
import numpy as np

In [17]:
df = pd.read_csv("C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/Air Quality DataSets/combined_Clean_Data.csv")
df.head()

,Station_id,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),...,Ozone (ug/m3),Benzene (ug/m3),Toluene (ug/m3),Temp (degree C),RH (%),WS (m/s),WD (deg),SR (W/mt2),RF (mm),AT (degree C)
0,AP001,2016-07-01 10:00:00,2016-07-01 11:00:00,10.67,39.0,17.67,39.20,32.33,7.07,6.60,...,14.50,1.00,4.63,33.43,71.67,2.30,226.33,123.67,0.0,23.05
1,AP001,2016-07-01 11:00:00,2016-07-01 12:00:00,2.00,39.0,20.50,41.90,35.80,7.40,NaN,...,15.00,0.70,4.50,33.70,70.00,2.50,223.00,186.00,0.0,NaN
2,AP001,2016-07-01 14:00:00,2016-07-01 15:00:00,20.50,50.0,15.40,43.60,32.78,6.35,6.38,...,10.50,0.60,4.50,33.57,63.50,1.88,223.00,240.50,0.0,NaN
3,AP001,2016-07-01 15:00:00,2016-07-01 16:00:00,15.25,59.5,24.30,45.12,40.12,6.65,6.53,...,6.60,0.77,5.33,33.55,63.75,1.40,213.25,152.25,0.0,25.60
4,AP001,2016-07-01 16:00:00,2016-07-01 17:00:00,11.67,60.0,26.73,49.10,43.80,6.93,5.70,...,17.43,0.80,6.47,33.57,64.00,1.50,208.67,92.67,0.0,25.70


# Function to compute AQI sub-index

In [18]:
def calculate_sub_index(val, breakpoints):
    for bp in breakpoints:
        if bp['low'] <= val <= bp['high']:
            return ((bp['index_high'] - bp['index_low']) / (bp['high'] - bp['low'])) * (val - bp['low']) + bp['index_low']
    return np.nan  # If value is outside defined breakpoints


In [19]:
# Define breakpoints for each pollutant
breakpoints = {
    'PM2.5 (ug/m3)': [
        {'low': 0, 'high': 30, 'index_low': 0, 'index_high': 50},
        {'low': 31, 'high': 60, 'index_low': 51, 'index_high': 100},
        {'low': 61, 'high': 90, 'index_low': 101, 'index_high': 200},
        {'low': 91, 'high': 120, 'index_low': 201, 'index_high': 300},
        {'low': 121, 'high': 250, 'index_low': 301, 'index_high': 400},
        {'low': 251, 'high': 500, 'index_low': 401, 'index_high': 500},
    ],
    'PM10 (ug/m3)': [
        {'low': 0, 'high': 50, 'index_low': 0, 'index_high': 50},
        {'low': 51, 'high': 100, 'index_low': 51, 'index_high': 100},
        {'low': 101, 'high': 250, 'index_low': 101, 'index_high': 200},
        {'low': 251, 'high': 350, 'index_low': 201, 'index_high': 300},
        {'low': 351, 'high': 430, 'index_low': 301, 'index_high': 400},
        {'low': 431, 'high': 500, 'index_low': 401, 'index_high': 500},
    ],
    'NO2 (ug/m3)': [
        {'low': 0, 'high': 40, 'index_low': 0, 'index_high': 50},
        {'low': 41, 'high': 80, 'index_low': 51, 'index_high': 100},
        {'low': 81, 'high': 180, 'index_low': 101, 'index_high': 200},
        {'low': 181, 'high': 280, 'index_low': 201, 'index_high': 300},
        {'low': 281, 'high': 400, 'index_low': 301, 'index_high': 400},
        {'low': 401, 'high': 500, 'index_low': 401, 'index_high': 500},
    ],
    'SO2 (ug/m3)': [
        {'low': 0, 'high': 40, 'index_low': 0, 'index_high': 50},
        {'low': 41, 'high': 80, 'index_low': 51, 'index_high': 100},
        {'low': 81, 'high': 380, 'index_low': 101, 'index_high': 200},
        {'low': 381, 'high': 800, 'index_low': 201, 'index_high': 300},
        {'low': 801, 'high': 1600, 'index_low': 301, 'index_high': 400},
        {'low': 1601, 'high': 2000, 'index_low': 401, 'index_high': 500},
    ],
    'CO (mg/m3)': [
        {'low': 0.0, 'high': 1.0, 'index_low': 0, 'index_high': 50},
        {'low': 1.1, 'high': 2.0, 'index_low': 51, 'index_high': 100},
        {'low': 2.1, 'high': 10.0, 'index_low': 101, 'index_high': 200},
        {'low': 10.1, 'high': 17.0, 'index_low': 201, 'index_high': 300},
        {'low': 17.1, 'high': 34.0, 'index_low': 301, 'index_high': 400},
        {'low': 34.1, 'high': 50.0, 'index_low': 401, 'index_high': 500},
    ],
    'Ozone (ug/m3)': [
        {'low': 0, 'high': 50, 'index_low': 0, 'index_high': 50},
        {'low': 51, 'high': 100, 'index_low': 51, 'index_high': 100},
        {'low': 101, 'high': 168, 'index_low': 101, 'index_high': 200},
        {'low': 169, 'high': 208, 'index_low': 201, 'index_high': 300},
        {'low': 209, 'high': 748, 'index_low': 301, 'index_high': 400},
        {'low': 749, 'high': 1000, 'index_low': 401, 'index_high': 500},
    ]
}

In [20]:
# Calculate AQI for each row
def calculate_aqi(row):
    aqi_values = []
    for pollutant, bp_list in breakpoints.items():
        if pd.notnull(row.get(pollutant)):
            aqi = calculate_sub_index(row[pollutant], bp_list)
            if aqi is not None:
                aqi_values.append(aqi)
    return max(aqi_values) if aqi_values else np.nan

# Apply to your combined DataFrame
df['AQI'] = df.apply(calculate_aqi, axis=1)

# Preview
df[['Station_id', 'From Date', 'AQI']].head()

,Station_id,From Date,AQI
0,AP001,2016-07-01 10:00:00,49.000000
1,AP001,2016-07-01 11:00:00,52.130769
2,AP001,2016-07-01 14:00:00,54.266667
3,AP001,2016-07-01 15:00:00,59.500000
4,AP001,2016-07-01 16:00:00,61.176923


In [21]:
def classify_aqi(aqi):
    if pd.isna(aqi):
        return "Unknown"
    elif aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Satisfactory"
    elif aqi <= 200:
        return "Moderate"
    elif aqi <= 300:
        return "Poor"
    elif aqi <= 400:
        return "Very Poor"
    elif aqi <= 500:
        return "Severe"
    else:
        return "Hazardous"

# Apply classification
df['AQI_Category'] = df['AQI'].apply(classify_aqi)

# Optional: See AQI category distribution
print(df['AQI_Category'].value_counts())


AQI_Category
Satisfactory    179730
Moderate        164800
Good             85082
Very Poor        40144
Poor             39051
Unknown          14480
Severe           11780
Name: count, dtype: int64


In [22]:
df['From Date'] = pd.to_datetime(df['From Date'], errors = 'coerce')
df['To Date'] = pd.to_datetime(df['To Date'], errors = 'coerce')

In [23]:
df.head()

,Station_id,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),...,Toluene (ug/m3),Temp (degree C),RH (%),WS (m/s),WD (deg),SR (W/mt2),RF (mm),AT (degree C),AQI,AQI_Category
0,AP001,2016-07-01 10:00:00,2016-07-01 11:00:00,10.67,39.0,17.67,39.20,32.33,7.07,6.60,...,4.63,33.43,71.67,2.30,226.33,123.67,0.0,23.05,49.000000,Good
1,AP001,2016-07-01 11:00:00,2016-07-01 12:00:00,2.00,39.0,20.50,41.90,35.80,7.40,NaN,...,4.50,33.70,70.00,2.50,223.00,186.00,0.0,NaN,52.130769,Satisfactory
2,AP001,2016-07-01 14:00:00,2016-07-01 15:00:00,20.50,50.0,15.40,43.60,32.78,6.35,6.38,...,4.50,33.57,63.50,1.88,223.00,240.50,0.0,NaN,54.266667,Satisfactory
3,AP001,2016-07-01 15:00:00,2016-07-01 16:00:00,15.25,59.5,24.30,45.12,40.12,6.65,6.53,...,5.33,33.55,63.75,1.40,213.25,152.25,0.0,25.60,59.500000,Satisfactory
4,AP001,2016-07-01 16:00:00,2016-07-01 17:00:00,11.67,60.0,26.73,49.10,43.80,6.93,5.70,...,6.47,33.57,64.00,1.50,208.67,92.67,0.0,25.70,61.176923,Satisfactory


In [24]:
file_path = "C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/Air Quality DataSets/combined_Clean_Data.csv"
df.to_csv(file_path, index = False)